In [1]:
import pandas as pd
import numpy as np
import gc
import os, shutil
import sys
import matplotlib.pyplot as plt
import open3d as o3d
import transforms3d
from tqdm import tqdm_notebook
import copy
from multiprocessing import Pool, cpu_count

In [2]:
PATH = "/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/testing"
BASE_FOLDER = "dynamic"

OUTPUT_STATIC_FOLDER  = "_corr_static"
OUTPUT_DYNAMIC_FOLDER = "_corr_dynamic"
PAIR_FILE = "pair_with_pose_transform_dynseg_testing.csv"


BASE_PATH = os.path.join(PATH, BASE_FOLDER)

In [3]:
pair_path = os.path.join(PATH, PAIR_FILE)
df_pair = pd.read_csv(pair_path)
df_pair.shape

(30390, 11)

In [4]:
df_pair.sample()

,static_path,dynamic_path,pose_x,pose_y,pose_yaw,static_x,static_y,static_yaw,dynamic_x,dynamic_y,dynamic_yaw
12195,/home/saby/Projects/ati/data/data/datasets/Car...,/home/saby/Projects/ati/data/data/datasets/Car...,-0.194972,-0.113953,-3.970318,7.752102,330.900299,3.694073,7.55713,330.786346,-0.276245


In [5]:
# Function to get transformation matrix for a given pose
def pose2matrix(translation_list, rotation_angle_list, zoom_list=[1,1,1]):
    # trans_vec = np.array(translation_list)
    rot_ang = [np.deg2rad(ang) for ang in rotation_angle_list ]
    rot_mat = transforms3d.euler.euler2mat(rot_ang[0], rot_ang[1], rot_ang[2])
    zoom_vec = np.array(zoom_list)
    # transform_mat = transforms3d.affines.compose(trans_vec, rot_mat, zoom_vec)
    transform_mat = transforms3d.affines.compose(translation_list, rot_mat, zoom_list)
    return transform_mat

def getint(name):
    return int(name.split('.')[0])

In [6]:
# def transform_corr_static_pcd(static_pcd, pair_row):
#     # Calculate all the transforms
#     transform_lidar   = pose2matrix([0, 0, 0],
#                                     [0, 0, 90],
#                                     [1, 1, -1])
#     transform_static1  = pose2matrix([pair_row['static_x'], pair_row['static_y'], pair_row['static_z']],
#                                     [pair_row['static_roll'],pair_row['static_pitch'], pair_row['static_yaw']])
#     transform_dynamic1 = pose2matrix([pair_row['dynamic_x'], pair_row['dynamic_y'], pair_row['dynamic_z']],
#                                     [pair_row['dynamic_roll'], pair_row['dynamic_pitch'], pair_row['dynamic_yaw']])
#     transform1 = np.matmul(np.linalg.inv(transform_dynamic1), transform_static1)
    
#     transform_static2  = pose2matrix([0, 0, 0],
#                                     [pair_row['static_roll'],pair_row['static_pitch'], pair_row['static_yaw']])
#     transform_dynamic2 = pose2matrix([0, 0, 0],
#                                     [pair_row['dynamic_roll'], pair_row['dynamic_pitch'], pair_row['dynamic_yaw']])
#     transform2 = np.matmul(np.linalg.inv(transform_dynamic2), transform_static2)
    
#     # Apply all the transforms
#     static_pcd_tmp = copy.deepcopy(static_pcd)
#     static_pcd_tmp.transform(np.linalg.inv(transform_lidar))
#     static_pcd_tmp.transform(np.linalg.inv(transform1))
#     static_pcd_tmp.transform(transform2)
#     static_pcd_tmp.transform(transform2)     # Need to fix this hack
#     static_pcd_tmp.transform(transform_lidar)
    
#     return static_pcd_tmp

def transform_corr_static_pcd(static_pcd, pair_row):
    # Calculate all the transforms
    transform_lidar   = pose2matrix([0, 0, 0],
                                    [0, 0, 90],
                                    [1, 1, -1])
    transform_static1  = pose2matrix([pair_row['static_x'], pair_row['static_y'], 0],
                                    [0,0, pair_row['static_yaw']])
    transform_dynamic1 = pose2matrix([pair_row['dynamic_x'], pair_row['dynamic_y'], 0],
                                    [0, 0, pair_row['dynamic_yaw']])
    transform1 = np.matmul(np.linalg.inv(transform_dynamic1), transform_static1)
    
    transform_static2  = pose2matrix([0, 0, 0],
                                    [0,0, pair_row['static_yaw']])
    transform_dynamic2 = pose2matrix([0, 0, 0],
                                    [0, 0, pair_row['dynamic_yaw']])
    transform2 = np.matmul(np.linalg.inv(transform_dynamic2), transform_static2)
    
    # Apply all the transforms
    static_pcd_tmp = copy.deepcopy(static_pcd)
    static_pcd_tmp.transform(np.linalg.inv(transform_lidar))
    static_pcd_tmp.transform(np.linalg.inv(transform1))
    static_pcd_tmp.transform(transform2)
    static_pcd_tmp.transform(transform2)     # Need to fix this hack
    static_pcd_tmp.transform(transform_lidar)
    
    return static_pcd_tmp

In [7]:
# for DYNAMICSUBFOLDER in os.listdir(os.path.join(BASE_PATH)):
#     OUTPUT_STATIC_PATH = os.path.join(BASE_PATH, DYNAMICSUBFOLDER, STATIC_FOLDER)
#     if not os.path.exists(OUTPUT_STATIC_PATH):
#         os.makedirs(OUTPUT_STATIC_PATH)
#     else:
#         shutil.rmtree(OUTPUT_STATIC_PATH)
#         os.makedirs(OUTPUT_STATIC_PATH)
        
#     OUTPUT_DYNAMIC_PATH = os.path.join(BASE_PATH, DYNAMICSUBFOLDER, OUTPUT_DYNAMIC_FOLDER)
#     if not os.path.exists(OUTPUT_DYNAMIC_PATH):
#         os.makedirs(OUTPUT_DYNAMIC_PATH)
#     else:
#         shutil.rmtree(OUTPUT_DYNAMIC_PATH)
#         os.makedirs(OUTPUT_DYNAMIC_PATH)

In [8]:
def parallel_thread(arg):
    DYNAMICSUBFOLDER = arg
    
    OUTPUT_STATIC_PATH  = os.path.join(BASE_PATH, DYNAMICSUBFOLDER,  OUTPUT_STATIC_FOLDER)
    if not os.path.exists(OUTPUT_STATIC_PATH):
        os.makedirs(OUTPUT_STATIC_PATH)
    else:
        shutil.rmtree(OUTPUT_STATIC_PATH)
        os.makedirs(OUTPUT_STATIC_PATH)

    OUTPUT_DYNAMIC_PATH = os.path.join(BASE_PATH, DYNAMICSUBFOLDER, OUTPUT_DYNAMIC_FOLDER)
    if not os.path.exists(OUTPUT_DYNAMIC_PATH):
        os.makedirs(OUTPUT_DYNAMIC_PATH)
    else:
        shutil.rmtree(OUTPUT_DYNAMIC_PATH)
        os.makedirs(OUTPUT_DYNAMIC_PATH)
    
    file_idx = 0
    for idx, row in df_pair.iterrows():
        if DYNAMICSUBFOLDER == row['dynamic_path'].split("/")[-3]:
            fname = str(file_idx) + ".ply"
            static_src = row['static_path']
            static_dst = os.path.join(OUTPUT_STATIC_PATH, fname)
            
            static_pcd = o3d.io.read_point_cloud(static_src)
            static_pcd = transform_corr_static_pcd(static_pcd, row)
            o3d.io.write_point_cloud(static_dst, static_pcd, write_ascii=True)
            
            dynamic_src = row['dynamic_path']
            dynamic_dst = os.path.join(OUTPUT_DYNAMIC_PATH, fname)
            shutil.copyfile(dynamic_src, dynamic_dst)
            file_idx += 1
#             if file_idx == 20:
#                 break

In [9]:
parallel_args = os.listdir(os.path.join(BASE_PATH))
process_pool = Pool(cpu_count()-1)
# process_pool = Pool(10)
__ = [each for each in tqdm_notebook(process_pool.imap(parallel_thread,
                                                       parallel_args), total=len(parallel_args))]
process_pool.terminate()
gc.collect()

/home/saby/anaconda3/envs/ati/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


12

In [10]:
def draw_registration_result(src_pcd, dst_pcd, x_pt, y_pt, theta):    
    src_pcd_tmp = copy.deepcopy(src_pcd)
    dst_pcd_tmp = copy.deepcopy(dst_pcd)
    
    src_pcd_tmp.paint_uniform_color([1, 0, 0])  # red source
    dst_pcd_tmp.paint_uniform_color([0, 0, 0])  # blue target
    
    transform_mat = pose2matrix([x_pt, y_pt, 0], [0,0,theta])
    dst_pcd_tmp.transform(transform_mat)
    
    visualizer = o3d.JVisualizer()
    visualizer.add_geometry(src_pcd_tmp)
    visualizer.add_geometry(dst_pcd_tmp)
    visualizer.show()

In [11]:
DYNAMICSUBFOLDER = np.random.choice(os.listdir(os.path.join(BASE_PATH)))
OUTPUT_STATIC_PATH  = os.path.join(BASE_PATH, DYNAMICSUBFOLDER,  OUTPUT_STATIC_FOLDER)
OUTPUT_DYNAMIC_PATH = os.path.join(BASE_PATH, DYNAMICSUBFOLDER, OUTPUT_DYNAMIC_FOLDER)
some_idx = np.random.choice(len(os.listdir(OUTPUT_STATIC_PATH)))
some_fname = str(some_idx) + ".ply"
# some_row = df_pair[(df_pair['dynamic_path'].split("/")[-3] == DYNAMICSUBFOLDER) &\
#             (df_pair['dynamic_path'].split("/")[-1] == some_fname)]

some_static_dst = os.path.join(OUTPUT_STATIC_PATH, some_fname)
some_dynamic_dst = os.path.join(OUTPUT_DYNAMIC_PATH, some_fname)

some_static_pcd = o3d.io.read_point_cloud(some_static_dst)
some_dynamic_pcd = o3d.io.read_point_cloud(some_dynamic_dst)

# print("Pose_diff_x: {} | Pose_diff_y: {} | Pose_diff_yaw: {} |".format(some_row['pose_x'], some_row['pose_y'], some_row['pose_yaw']))
print(some_dynamic_dst, some_static_dst)
draw_registration_result(some_static_pcd, some_dynamic_pcd, 0, 0, 0)

/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/testing/dynamic/23/_corr_dynamic/4900.ply /home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/testing/dynamic/23/_corr_static/4900.ply


JVisualizer with 2 geometries